# Sentiment Analysis

In [1]:
!pip install konlpy > /dev/null

In [2]:
import pandas as pd
import numpy as np

In [3]:
train_df = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', sep = '\t')
test_df = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt', sep = '\t')
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [4]:
train_df.shape, test_df.shape

((150000, 3), (50000, 3))

## Preprocessing

### Train Dataset

In [5]:
train_df.document.nunique()

146182

In [6]:
train_df.isnull().sum()

id          0
document    5
label       0
dtype: int64

In [7]:
train_df.dropna(how = 'any', inplace = True)
train_df.shape

(149995, 3)

In [8]:
train_df.drop_duplicates(subset = ['document'], inplace = True)
train_df.shape

(146182, 3)

In [9]:
train_df.label.value_counts()

0    73342
1    72840
Name: label, dtype: int64

### Test Dataset

In [10]:
test_df.document.nunique()

49157

In [11]:
test_df.isnull().sum()

id          0
document    3
label       0
dtype: int64

In [12]:
test_df.dropna(how = 'any', inplace = True)
test_df.shape

(49997, 3)

In [13]:
test_df.drop_duplicates(subset = ['document'], inplace = True)
test_df.shape

(49157, 3)

In [14]:
test_df.label.value_counts()

1    24711
0    24446
Name: label, dtype: int64

## Text Preprocessing

### Train Dataset

In [15]:
train_df['document'] = train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '')
train_df.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


In [16]:
train_df['document'].replace('', np.nan, inplace = True)
train_df.document.isnull().sum()

391

In [17]:
train_df.dropna(how = 'any', inplace = True)
train_df.shape

(145791, 3)

### Test Dataset

In [18]:
test_df['document'] = test_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '')

In [19]:
test_df['document'].replace('', np.nan, inplace = True)

In [20]:
test_df.dropna(how = 'any', inplace = True)
test_df.shape

(48995, 3)

## Korean Processing

In [21]:
from konlpy.tag import Okt
okt = Okt()

In [22]:
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving 한국어불용어100.txt to 한국어불용어100.txt


In [23]:
df = pd.read_csv(filename, sep = '\s+', header = None)
df.head()

,0,1,2
0,이,VCP,0.018280
1,있,VA,0.011699
2,하,VV,0.009774
3,것,NNB,0.009733
4,들,XSN,0.006898


### 품사 무시 중복 배제

In [24]:
stopwords = set(df[0])
len(stopwords)

94

In [25]:
text = '교도소 이야기구먼 솔직히 재미는 없다평점 조정   '
okt.morphs(text)

['교도소', '이야기', '구먼', '솔직히', '재미', '는', '없다', '평점', '조정']

In [26]:
okt.morphs(text, stem = True)

['교도소', '이야기', '구먼', '솔직하다', '재미', '는', '없다', '평점', '조정']

In [27]:
' '.join(okt.morphs(text, stem = True))

'교도소 이야기 구먼 솔직하다 재미 는 없다 평점 조정'

In [29]:
from tqdm.notebook import tqdm
str_train = []
for sentence in tqdm(train_df.document):
    morphs = okt.morphs(sentence.strip(), stem = True)
    temp_str = ' '.join(word for word in morphs if word not in stopwords)
    str_train.append(temp_str)

  0%|          | 0/145791 [00:00<?, ?it/s]

In [30]:
str_train[:5]

['아 더빙 진짜 짜증나다 목소리',
 '흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍다 않다',
 '너 무재 밓었 다그 래서 보다 추천 다',
 '교도소 이야기 구먼 솔직하다 재미 는 없다 평점 조정',
 '사이 몬페 의 익살스럽다 연기 돋보이다 영화 스파이더맨 에서 늙다 보이다 하다 커스틴 던스트 너무나도 이쁘다 보이다']

In [31]:
str_test = []
for sentence in tqdm(test_df.document):
    morphs = okt.morphs(sentence.strip(), stem = True)
    temp_str = ' '.join(word for word in morphs if word not in stopwords)
    str_test.append(temp_str)

  0%|          | 0/48995 [00:00<?, ?it/s]

In [32]:
str_test[:5]

['굳다 ㅋ',
 '뭐 야 평점 은 나쁘다 않다 짜다 리 는 더욱 아니다',
 '지루하다 않다 완전 막장 임 돈 주다 보기 에는',
 '만 아니다 별 다섯 주다 왜 로 나오다 제 심기 를 불편하다 하다',
 '음악 주가 되다 최고 의 음악 영화']

In [33]:
y_train = train_df.label.values
y_test = test_df.label.values

## Feature Trans

### Count Vectorizer

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer()

In [36]:
cvect.fit(str_train)
X_train = cvect.transform(str_train)
X_test = cvect.transform(str_test)
X_train.shape, X_test.shape

((145791, 42092), (48995, 42092))

## 모델 학습/예측/평가

In [37]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [39]:
lr = LogisticRegression(max_iter = 500)
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [40]:
pred = lr.predict(X_test)

In [41]:
accuracy_score(y_test, pred)

0.8259006021022554

## 실제 테스트

In [48]:
review1 = '솔직히 난 그냥 재밋엇다... 가볍게 웃고 ... 살짝 울기도 햇는데...요새 힘든일이 많아서 그런지...상영 시간만큼은 잠깐 잊어버리고 너무 재밋게 봣네요..'
review2 = '두서도 없고,맥락도 없고,감동도 없고,유머만 조금,우주전쟁영화도 개연성이 있는데, 물통타고 올라오는 개연성없는 맥락의 영화,정말 돈아깝다'

In [43]:
review2.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '')

'두서도 없고,맥락도 없고,감동도 없고,유머만 조금,우주전쟁영화도 개연성이 있는데, 물통타고 올라오는 개연성없는 맥락의 영화,정말 돈아깝다'

In [44]:
import re
review1 = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', review1)
review2 = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', review2)
review1, review2

('솔직히 난 그냥 재밋엇다 가볍게 웃고  살짝 울기도 햇는데요새 힘든일이 많아서 그런지상영 시간만큼은 잠깐 잊어버리고 너무 재밋게 봣네요',
 '두서도 없고맥락도 없고감동도 없고유머만 조금우주전쟁영화도 개연성이 있는데 물통타고 올라오는 개연성없는 맥락의 영화정말 돈아깝다')

In [49]:
reviews = [review1, review2]
review_list = []
for review in reviews:
    review = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', review)
    morphs = okt.morphs(review.strip(), stem = True)
    temp_str = ' '.join(word for word in morphs if word not in stopwords)
    review_list.append(temp_str)

In [50]:
review_list

['솔직하다 난 그냥 재밋엇다 가볍다 웃다 살짝 울 기도 햇 늘다 요새 힘드다 일이 많다 상영 만큼은 잠깐 잊어버리다 너무 재밋 게 봣네 요',
 '두서 도 없다 맥락 도 없다 감동 도 없다 유머 만 조금 우주전쟁 영화 도 개연 성 있다 물통 타고 올라오다 개연 성 없다 맥락 의 영화 정말 돈 아깝다']

In [51]:
review_cv = cvect.transform(review_list)
pred = lr.predict(review_cv)
pred

array([1, 0])